**Cài đặt thư viện UnderTheSea để hỗ trợ tách từ trong tiếng Việt. Xem thêm các tính năng của thư viện tại: https://github.com/undertheseanlp/underthesea**

In [1]:
!pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.0 MB 23.5 MB/s 
     |████████████████████████████████| 1.0 MB 99.5 MB/s 
     |████████████████████████████████| 591 kB 67.2 MB/s 


**Import các thư viện cần thiết**

In [2]:
import os
import re
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from underthesea import text_normalize
from underthesea import word_tokenize
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

**Lấy dữ liệu các tài liệu/văn bản mẫu từ**: https://github.com/HUTECH-OpenCourseWare/IRS.git

In [3]:
!git clone https://github.com/HUTECH-OpenCourseWare/IRS.git

Cloning into 'IRS'...
remote: Enumerating objects: 271, done.
remote: Counting objects: 100% (271/271), done.
remote: Compressing objects: 100% (271/271), done.
remote: Total 271 (delta 0), reused 271 (delta 0), pack-reused 0
Receiving objects: 100% (271/271), 441.48 KiB | 1.24 MiB/s, done.


**Đọc dữ liệu các tài liệu/văn bản từ các files chứa trong folder "/content/IRS_Course/data/vnexpress/<chủ_đề>"**

In [4]:
topic = 'the-thao'
data_root_dir_path = '/content/IRS/data/vnexpress/{}'.format(topic)
docs = []
for file_name in os.listdir(data_root_dir_path):
  file_path = os.path.join(data_root_dir_path, file_name)
  lines = []
  with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
      line = line.lower().strip()
      lines.append(line)

  text = " ".join(lines)
  clean_text = re.sub('\W+',' ', text)
  docs.append(clean_text)
print(f'Hoàn tất đọc tất cả [{len(docs)}] tài liệu/văn bản - chủ đề: [{topic}]')

Hoàn tất đọc tất cả [39] tài liệu/văn bản - chủ đề: [the-thao]


**Tiến hành các bước xử lý & tách từ cho văn bản tiếng Việt**

In [5]:
# Tạo một cấu trúc danh sách - chứa dữ liệu đã được xử lý
processed_docs = []

# Đọc danh sách các từ stopwords trong tiếng Việt
vn_stopwords_file_path = '/content/IRS/data/stopwords/vietnamese-stopwords.txt'
stopwords = []
with open(vn_stopwords_file_path, 'r', encoding='utf-8') as f:
  for line in f:
    line = line.strip()
    stopwords.append(line)

# Viết hàm tiền xử lý và tách từ tiếng Việt
def preprocess(doc):
  # Tiến hành xử lý các lỗi từ/câu, dấu câu, v.v. trong tiếng Việt với hàm text_normalize
  normalized_doc = text_normalize(doc)
  # Tiến hành tách từ
  tokens = word_tokenize(normalized_doc)
  # Lọc stopwords
  non_stopword_tokens = []
  for token in tokens:
    if token not in stopwords:
      non_stopword_tokens.append(token)
  # Tiến hành kết hợp các từ ghép trong tiếng Việt bằng '_'
  combined_tokens = [token.replace(' ', '_') for token in non_stopword_tokens]
  return (normalized_doc, combined_tokens)

for doc in docs:
  (normalized_doc, combined_tokens) = preprocess(doc)
  processed_docs.append((normalized_doc, combined_tokens))

**Tiến hành thử nghiệm học biểu diễn các từ trong tập văn bản [processed_docs] với thư viện Word2Vec của Gensim**

In [6]:
sentences = []
for (normalized_doc, combined_tokens) in processed_docs:
  sentences.append(combined_tokens)
word2vec_model = Word2Vec(sentences=sentences, 
                          size=100, 
                          window=5, 
                          min_count=1, 
                          workers=1)
print('Hoàn tất quá trình huấn luyện mô hình Word2Vec cho tập dữ liệu !')

Hoàn tất quá trình huấn luyện mô hình Word2Vec cho tập dữ liệu !


**Xuất ra danh sách các từ có trong tập từ vựng**

In [7]:
vocabs = [token for token in word2vec_model.wv.vocab]
print(vocabs)

['liverpool', 'nhì', 'bảng', 'champions', 'league', 'anhthắng', 'napoli', '2', '0', 'lượt', 'thầy_trò', 'jurgen', 'klopp', 'giật', 'đỉnh', 'a', 'đối_thủ', 'tối', '1', '11', '15', 'thắng', 'thua', 'xếp', 'hiệu_số', 'bàn', 'thắng_bại', 'hai', 'đối_đầu', 'giúp', 'đội', 'bóng', 'italy', 'vòng', '8', 'nguy_cơ', 'bayern', 'real', 'psg', 'salah', 'tỷ_số', 'phút', '85', 'ảnh', 'epa', 'cách_biệt', 'bốn', 'chiếm', 'hlv', 'tiền_đạo', 'darwin', 'nunez', 'sân', 'đầu', 'đắt', 'lịch_sử', 'cầu_thủ', 'đá', 'trận', 'leeds', 'cạnh', 'andy', 'robertson', 'harvey', 'elliott', 'joe', 'gomez', 'curtis', 'jones', 'bố_trí', 'trái', 'thay', 'sơ_đồ', '4', '3', 'khai', 'tràn', 'đường', 'chọc', 'khe', 'sút', 'vọt', 'xà', '29', 'pha', 'phản_công_mẫu_mực', 'thiago', 'chân', 'dứt_điểm', 'tuyến', 'thủ_thành', 'alex', 'meret', 'tung', 'cản_phá', 'đánh_đầu', 'chia', 'vui', 'lập', 'công', 'ap', 'e_dè', 'anfield', 'đột_phá', 'khvicha', 'kvaratskhelia', 'cánh', 'hiệp', 'khán_đài', 'im_bặt', 'leo', 'ostigard', 'bay', 'tung_

**Thử nghiệm chức năng tìm kiếm các từ gần/tương đồng/hay xuất hiện chung với một từ**

In [8]:
sims = word2vec_model.wv.most_similar('cầu_thủ', topn=10)
print(sims)

[('đội', 0.9426406025886536), ('phút', 0.9389937520027161), ('trận', 0.9362512826919556), ('tiền_đạo', 0.9331744313240051), ('giải', 0.9267739653587341), ('1', 0.9267497658729553), ('11', 0.9260605573654175), ('đấu', 0.9260364770889282), ('2022', 0.9250099658966064), ('nam', 0.9242852330207825)]


**Tiến hành thử nghiệm học biểu diễn các từ trong tập văn bản [processed_docs] với thư viện Doc2Vec của Gensim**

In [9]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(sentences)]
doc2vec_model = Doc2Vec(documents,
                        vector_size=100, 
                        window=2, 
                        min_count=1, 
                        workers=1)

**Thử lấy vector biểu diễn của tài liệu đầu tiên**

In [10]:
doc2vec_model.docvecs[0]

array([-0.02286145, -0.00688301,  0.05733623, -0.03763993,  0.02754937,
        0.04620416, -0.01186034, -0.03283711,  0.01729831, -0.0237403 ,
       -0.04368305,  0.00599364, -0.02098591, -0.05552647,  0.04374911,
        0.03329016, -0.06928718,  0.01086011, -0.01513694, -0.00140159,
        0.01250541,  0.01099868, -0.01011164,  0.05714671,  0.01052094,
       -0.00803035,  0.04655482, -0.02180494,  0.03740108, -0.02887356,
       -0.00332454,  0.01340596, -0.0395426 , -0.03662034,  0.03535185,
        0.0088013 , -0.03998219,  0.05006882, -0.0760505 , -0.01420349,
        0.02326053, -0.02911623, -0.00084083,  0.05177208,  0.01160359,
       -0.00916797,  0.00482333,  0.04069285,  0.01178363, -0.05773144,
        0.01082948, -0.02480627,  0.05463095,  0.00281873, -0.02291851,
        0.05105351,  0.00391827,  0.00988708, -0.0231679 , -0.03078115,
       -0.00640672,  0.05401685, -0.06322952,  0.02924149, -0.02784145,
        0.00924785,  0.00704862, -0.05692776, -0.00453266,  0.04